In [11]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import requests
import gmaps

In [2]:
city_data_df = pd.read_csv("../weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hours),Snow inches (last 3 hours)
0,0,Bambous Virieux,MU,2021-06-22 03:37:43,-20.3428,57.7575,66.45,88,75,3.44,broken clouds,0,0
1,1,Port Elizabeth,ZA,2021-06-22 03:37:43,-33.9180,25.5701,71.89,28,0,9.22,clear sky,0,0
2,2,Steamboat Springs,US,2021-06-22 03:39:55,40.4850,-106.8317,71.19,45,1,6.91,clear sky,0,0
3,3,New Norfolk,AU,2021-06-22 03:37:25,-42.7826,147.0587,50.76,65,100,1.74,overcast clouds,0,0
4,4,Ushuaia,AR,2021-06-22 03:37:30,-54.8000,-68.3000,35.26,75,0,11.50,clear sky,0,0


In [3]:
cols = city_data_df.columns.tolist()

In [4]:
max_temp = float(input("maximum temperature? "))
min_temp = float(input("minimum temperature? "))
rain = str(input('raining? (yes/no)\n'))
snow = str(input('snowing? (yes/no)\n'))

maximum temperature? 85
minimum temperature? 60
raining? (yes/no)
no
snowing? (yes/no)
no


In [5]:
for index, row in city_data_df.iterrows():
    if rain == "no":
        rain_inches = city_data_df["Rain inches (last 3 hours)"] == 0
    else:
        rain_inches = city_data_df["Rain inches (last 3 hours)"] != 0
    if snow == "no":
        snow_inches = city_data_df["Snow inches (last 3 hours)"] == 0
    else:
        snow_inches = city_data_df["Snow inches (last 3 hours)"] != 0

In [9]:

# Filter the dataset to find the cities that fit the criteria.

preferred_cities_df = city_data_df.loc[(city_data_df["Rain inches (last 3 hours)"] != rain_inches) &
                                       (city_data_df["Snow inches (last 3 hours)"] != snow_inches) &
                                       (city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                       
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hours),Snow inches (last 3 hours)
0,0,Bambous Virieux,MU,2021-06-22 03:37:43,-20.3428,57.7575,66.45,88,75,3.44,broken clouds,0,0
1,1,Port Elizabeth,ZA,2021-06-22 03:37:43,-33.9180,25.5701,71.89,28,0,9.22,clear sky,0,0
2,2,Steamboat Springs,US,2021-06-22 03:39:55,40.4850,-106.8317,71.19,45,1,6.91,clear sky,0,0
5,5,Bradu,RO,2021-06-22 03:39:55,44.7833,24.9000,64.02,89,2,3.44,clear sky,0,0
9,9,Kirakira,SB,2021-06-22 03:39:56,-10.4544,161.9205,83.64,77,96,10.60,overcast clouds,0,0
11,11,Vaini,TO,2021-06-22 03:38:12,-21.2000,-175.2000,80.76,78,20,12.66,few clouds,0,0
13,13,East London,ZA,2021-06-22 03:37:48,-33.0153,27.9116,67.14,32,0,6.91,clear sky,0,0
16,16,Rikitea,PF,2021-06-22 03:37:29,-23.1203,-134.9692,75.51,82,89,13.53,overcast clouds,0,0
17,17,Fairbanks,US,2021-06-22 03:39:57,64.8378,-147.7164,67.01,81,75,4.61,broken clouds,0,0
18,18,Faanui,PF,2021-06-22 03:38:29,-16.4833,-151.7500,79.92,73,0,7.45,clear sky,0,0


In [10]:
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bambous Virieux,MU,66.45,broken clouds,-20.3428,57.7575,
1,Port Elizabeth,ZA,71.89,clear sky,-33.9180,25.5701,
2,Steamboat Springs,US,71.19,clear sky,40.4850,-106.8317,
5,Bradu,RO,64.02,clear sky,44.7833,24.9000,
9,Kirakira,SB,83.64,overcast clouds,-10.4544,161.9205,
11,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,
13,East London,ZA,67.14,clear sky,-33.0153,27.9116,
16,Rikitea,PF,75.51,overcast clouds,-23.1203,-134.9692,
17,Fairbanks,US,67.01,broken clouds,64.8378,-147.7164,
18,Faanui,PF,79.92,clear sky,-16.4833,-151.7500,


In [12]:

params = {"radius": 4000,"type": "lodging","key": "28fc190ab29499f2c20cd1b7b2206c5a"}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotels = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("hotel not found... skipping")

hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... s

hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... s

In [14]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [15]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
figure = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(heat_layer)
figure.add_layer(marker_layer)

figure

Figure(layout=FigureLayout(height='420px'))

In [16]:
output_data_file = "WeatherPy_vacation.csv"
hotel_df.to_csv(output_data_file, index_label="city_id")